Group-10


Nency Badiyani  

Gajendra Muley 

Avinash Gupta 

harshal Dhuri


In [1]:
import nltk
import pandas as pd
import requests
from bs4 import BeautifulSoup
lemmatizer=nltk.stem.WordNetLemmatizer()
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from selenium import webdriver
import time
stop_words = stopwords.words('english')
import string
punctuations = list(string.punctuation)
import tkinter as tk
from tkinter import *

global ingredient_ans
global ingredients
global ing_questions
global ingredients_dict
global links

lst_set_keys=['quantity','units']
typ_keys=['type','form','sort','kind','variety','category']

def show_link():
    dish=Entry_recipe.get()
    query ="https://www.food.com/search/"
    query=query+str(dish)
    driver = webdriver.Chrome('./chromedriver') 
    driver.get(query)
    html = driver.page_source
    links=[]
    soup=BeautifulSoup(html,'html.parser')
    for a in soup.find_all('h2',class_='title'):
        links.append(a.find('a').get('href'))
    for l in links:
        print(l)
        
    
    clicked.set("select recipe")    
    dropdown_searched_list = OptionMenu(root,clicked,*links)       

    dropdown_searched_list.grid(row=3,column=2)

def url():
    url=clicked.get()
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    return soup

        

def get_recipe():
    soup=url()
    quant=[]
    details=[]

    a1=soup.find("ul",class_="ingredient-list svelte-1apyilh")
    for i in a1.find_all('li'):
        a=i.find('span',class_='ingredient-quantity svelte-1apyilh')
        quant.append(a.text)
    

    a2=soup.find("ul",class_="ingredient-list svelte-1apyilh")
    for i in a2.find_all('li'):
        a=i.find('span',class_='ingredient-text svelte-1apyilh')
        details.append(a.text.removeprefix('\n').removesuffix('\n'))
    ingredients={}

    for detail in details :
        for i in quant :
            ingredients[detail]=i
            quant.remove(i)
            break
    ingredient_ans.delete(1.0,tk.END)
    
    for k,v in ingredients.items():
        ingredient_ans.insert(tk.END,v + " : " + k)
        ingredient_ans.insert(tk.END,'\n')
       
    ingredients_dict={}
    units =['cup','teaspoon','l','tablespoon','gallon','mg','g','ml','kg','ounce','cups','package','tablespoons','teaspoons']

    for k,v in ingredients.items():
        
        words=word_tokenize(k)
        # print(words)
        words=[word for word in words if word not in stop_words]
        words=[word for word in words if word not in punctuations]
        pos_tagged_text=nltk.pos_tag(words)
        # print(pos_tagged_text)
        lst={}
        lst['quantity']=v
        for k in pos_tagged_text:
            if k[1]=='VBD' or k[1]=='VBN' or k[1]=='VB':
                lst['type']=k[0]

        for i, tuple in enumerate(pos_tagged_text):

            if tuple[1]=='NN'or tuple[1]=='NNS':
                
                if tuple[0] in units:
                    lst['units']=tuple[0]
                else:
            
                    try:
                        if pos_tagged_text[i+1][1]=='NN' or pos_tagged_text[i+1][1]=='NNS':
                            ingredients_dict[tuple[0]+' '+pos_tagged_text[i+1][0]]=lst
                        else:
                            ingredients_dict[tuple[0]]=lst

                 
                    except:
                        ingredients_dict[tuple[0]]=lst

    ing_questions=ingredient_q.get()
    ing_questions=str(ing_questions)
    ing_questions=word_tokenize(ing_questions)
    ing_questions=[lemmatizer.lemmatize(word,'n') for word in ing_questions]
    ing_questions=[word for word in ing_questions if word not in stop_words]
    ing_questions=[word for word in ing_questions if word not in punctuations]
    ing_questions = nltk.pos_tag(ing_questions)
    lst_set_keys=['quantity','units']
    typ_keys=['type','form','sort','kind','variety','category']
    for i in ing_questions:    
        if i[0] in lst_set_keys:            
            quan(ing_questions,ingredients_dict)
            break
        elif i[0] in typ_keys:            
            type_ing(ing_questions,ingredients_dict)
            break
        else:
            quan(ing_questions,ingredients_dict)

def quan(ing_questions,ingredients_dict):
    for w in ing_questions:
        if w[0] not in lst_set_keys and w[1]=='NNS' or w[1]=='NN':
            try:
                x = ingredients_dict[w[0]]
                try:
                    i=list(x.keys())[0] +' of ' + w[0] + ' is '+ list(x.values())[0] +x['units']
                    print(i)
                    ingredient_ans.delete(1.0,tk.END)
                    ingredient_ans.insert(tk.END,i)
                except:
                    i=list(x.keys())[0] +' of '+ w[0] +' is '+ list(x.values())[0]
                    print(i)
                    ingredient_ans.delete(1.0,tk.END)
                    ingredient_ans.insert(tk.END,i)
            except:
                ingredient_ans.delete(1.0,tk.END)
                ingredient_ans.insert(tk.END,"Please ask again unable to find ingredient")

def type_ing(ing_questions,ingredients_dict):
    for w in ing_questions:
        if w[0] not in typ_keys and w[1]=='NNS' or w[1]=='NN':
            try:
                x = ingredients_dict[w[0]]
                try:
                    i=list(x.keys())[1]+' of ' + w[0] +' is '+ x['type']
                    print(i)
                    ingredient_ans.delete(1.0,tk.END)
                    ingredient_ans.insert(tk.END,i)
                except:
                    i=list(x.keys())[1] + ' of ' +w[0] +' is '+list(x.values())[0]
                    print(i)
                    ingredient_ans.delete(1.0,tk.END)
                    ingredient_ans.insert(tk.END,i)
            except:
                ingredient_ans.delete(1.0,tk.END)
                ingredient_ans.insert(tk.END,"Please ask again unable to find ingredient")
next_steps=['next','after','subsequent','following','succeeding','upcoming','then','afterwards','thereafter']
prev_steps=['previous','before','preceding','prior','past','earlier']    

global directions_dict
def get_steps():
    soup=url()
    directions=[]
    for li in soup.find_all('li',class_="direction svelte-1apyilh"):
        directions.append(li.text.lower())
    directions_dict={}    
    for i in range (0,len(directions)):
        directions_dict[i+1]=directions[i]
    step_ans.delete(1.0,tk.END)
    for k,v in directions_dict.items():
        step_ans.insert(tk.END,str(k) + " : " + v)
        step_ans.insert(tk.END,'\n')
    
    d_dict={}
    for k , v in directions_dict.items():
        d_dict[k]=word_tokenize(v)
        d_dict[k]=[lemmatizer.lemmatize(word,'v') for word in d_dict[k]]
        d_dict[k]=[word for word in d_dict[k] if word not in stop_words]
        d_dict[k]=[word for word in d_dict[k] if word not in punctuations]
        # d_dict[k] = nltk.pos_tag(d_dict[k])
        
    count={}
    step_question=step_q.get()
    step_question=str(step_question)
    step_question=word_tokenize(step_question)
    step_question=[lemmatizer.lemmatize(word,'v') for word in step_question]
    step_question=[word for word in step_question if word not in punctuations]
    step_question = nltk.pos_tag(step_question)

    for i in step_question:
        for k,v in d_dict.items():
            if i[0].lower() in v :
                # print(i[0])
                try:           
                    count[k]=count[k]+1
                except:
                    count[k]=1
    
    for i in step_question:
        print(i[0])
    
        if i[0] in next_steps:
            n=next_step(count,directions_dict)
            print("n",n)
            step_ans.delete(1.0,tk.END)
            step_ans.insert(tk.END,n)
            # print(next_step(count))
            break
        elif i[0] in prev_steps:
            p=prev_step(count,directions_dict)

            step_ans.delete(1.0,tk.END)
            step_ans.insert(tk.END,p)
            # print(prev_step(count))
            break
        else:
            
            step_ans.delete(1.0,tk.END)
            step_ans.insert(tk.END,"please again ask question.")
    

def max_match(count):
    # print(count)
    
    return max(count , key = count.get)

def next_step(count,directions_dict):
    k=max_match(count)
    # print(k)
    # print(directions_dict[k+1])
    try:
        return directions_dict[k+1]
    except:
        return "This seems to be last step"

def prev_step(count,directions_dict):
    k=max_match(count)
    try:
        return directions_dict[k-1]
    except:
        return "this seems to be first step"





global ingredient_ans
global ingredients
global ing_questions
global ingredients_dict
global links

lst_set_keys=['quantity','units']
typ_keys=['type','form','sort','kind','variety','category']

def show_link():
    dish=Entry_recipe.get()
    query ="https://www.food.com/search/"
    query=query+str(dish)
    driver = webdriver.Chrome('./chromedriver') 
    driver.get(query)
    html = driver.page_source
    links=[]
    soup=BeautifulSoup(html,'html.parser')
    for a in soup.find_all('h2',class_='title'):
        links.append(a.find('a').get('href'))
    for l in links:
        print(l)
        
    
    clicked.set("select recipe")    
    dropdown_searched_list = OptionMenu(root,clicked,*links)       

    dropdown_searched_list.grid(row=3,column=2)

def url():
    url=clicked.get()
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    return soup

        

def get_recipe():
    soup=url()
    quant=[]
    details=[]

    a1=soup.find("ul",class_="ingredient-list svelte-1apyilh")
    for i in a1.find_all('li'):
        a=i.find('span',class_='ingredient-quantity svelte-1apyilh')
        quant.append(a.text)
    

    a2=soup.find("ul",class_="ingredient-list svelte-1apyilh")
    for i in a2.find_all('li'):
        a=i.find('span',class_='ingredient-text svelte-1apyilh')
        details.append(a.text.removeprefix('\n').removesuffix('\n'))
    ingredients={}

    for detail in details :
        for i in quant :
            ingredients[detail]=i
            quant.remove(i)
            break
    ingredient_ans.delete(1.0,tk.END)
    
    for k,v in ingredients.items():
        ingredient_ans.insert(tk.END,v + " : " + k)
        ingredient_ans.insert(tk.END,'\n')
       
    ingredients_dict={}
    units =['cup','teaspoon','l','tablespoon','gallon','mg','g','ml','kg','ounce','cups','package','tablespoons','teaspoons']

    for k,v in ingredients.items():
        
        words=word_tokenize(k)
        # print(words)
        words=[word for word in words if word not in stop_words]
        words=[word for word in words if word not in punctuations]
        pos_tagged_text=nltk.pos_tag(words)
        # print(pos_tagged_text)
        lst={}
        lst['quantity']=v
        for k in pos_tagged_text:
            if k[1]=='VBD' or k[1]=='VBN' or k[1]=='VB':
                lst['type']=k[0]

        for i, tuple in enumerate(pos_tagged_text):

            if tuple[1]=='NN'or tuple[1]=='NNS':
                
                if tuple[0] in units:
                    lst['units']=tuple[0]
                else:
            
                    try:
                        if pos_tagged_text[i+1][1]=='NN' or pos_tagged_text[i+1][1]=='NNS':
                            ingredients_dict[tuple[0]+' '+pos_tagged_text[i+1][0]]=lst
                        else:
                            ingredients_dict[tuple[0]]=lst

                 
                    except:
                        ingredients_dict[tuple[0]]=lst

    ing_questions=ingredient_q.get()
    ing_questions=str(ing_questions)
    ing_questions=word_tokenize(ing_questions)
    ing_questions=[lemmatizer.lemmatize(word,'n') for word in ing_questions]
    ing_questions=[word for word in ing_questions if word not in stop_words]
    ing_questions=[word for word in ing_questions if word not in punctuations]
    ing_questions = nltk.pos_tag(ing_questions)
    lst_set_keys=['quantity','units']
    typ_keys=['type','form','sort','kind','variety','category']
    for i in ing_questions:    
        if i[0] in lst_set_keys:            
            quan(ing_questions,ingredients_dict)
            break
        elif i[0] in typ_keys:            
            type_ing(ing_questions,ingredients_dict)
            break
        else:
            quan(ing_questions,ingredients_dict)

def quan(ing_questions,ingredients_dict):
    for w in ing_questions:
        if w[0] not in lst_set_keys and w[1]=='NNS' or w[1]=='NN':
            try:
                x = ingredients_dict[w[0]]
                try:
                    i=list(x.keys())[0] +' of ' + w[0] + ' is '+ list(x.values())[0] +x['units']
                    print(i)
                    ingredient_ans.delete(1.0,tk.END)
                    ingredient_ans.insert(tk.END,i)
                except:
                    i=list(x.keys())[0] +' of '+ w[0] +' is '+ list(x.values())[0]
                    print(i)
                    ingredient_ans.delete(1.0,tk.END)
                    ingredient_ans.insert(tk.END,i)
            except:
                ingredient_ans.delete(1.0,tk.END)
                ingredient_ans.insert(tk.END,"Please ask again unable to find ingredient")

def type_ing(ing_questions,ingredients_dict):
    for w in ing_questions:
        if w[0] not in typ_keys and w[1]=='NNS' or w[1]=='NN':
            try:
                x = ingredients_dict[w[0]]
                try:
                    i=list(x.keys())[1]+' of ' + w[0] +' is '+ x['type']
                    print(i)
                    ingredient_ans.delete(1.0,tk.END)
                    ingredient_ans.insert(tk.END,i)
                except:
                    i=list(x.keys())[1] + ' of ' +w[0] +' is '+list(x.values())[0]
                    print(i)
                    ingredient_ans.delete(1.0,tk.END)
                    ingredient_ans.insert(tk.END,i)
            except:
                ingredient_ans.delete(1.0,tk.END)
                ingredient_ans.insert(tk.END,"Please ask again unable to find ingredient")
next_steps=['next','after','subsequent','following','succeeding','upcoming','then','afterwards','thereafter']
prev_steps=['previous','before','preceding','prior','past','earlier']    

global directions_dict
def get_steps():
    soup=url()
    directions=[]
    for li in soup.find_all('li',class_="direction svelte-1apyilh"):
        directions.append(li.text.lower())
    directions_dict={}    
    for i in range (0,len(directions)):
        directions_dict[i+1]=directions[i]
    step_ans.delete(1.0,tk.END)
    for k,v in directions_dict.items():
        step_ans.insert(tk.END,str(k) + " : " + v)
        step_ans.insert(tk.END,'\n')
    
    d_dict={}
    for k , v in directions_dict.items():
        d_dict[k]=word_tokenize(v)
        d_dict[k]=[lemmatizer.lemmatize(word,'v') for word in d_dict[k]]
        d_dict[k]=[word for word in d_dict[k] if word not in stop_words]
        d_dict[k]=[word for word in d_dict[k] if word not in punctuations]
        # d_dict[k] = nltk.pos_tag(d_dict[k])
        
    count={}
    step_question=step_q.get()
    step_question=str(step_question)
    step_question=word_tokenize(step_question)
    step_question=[lemmatizer.lemmatize(word,'v') for word in step_question]
    step_question=[word for word in step_question if word not in punctuations]
    step_question = nltk.pos_tag(step_question)

    for i in step_question:
        for k,v in d_dict.items():
            if i[0].lower() in v :
                # print(i[0])
                try:           
                    count[k]=count[k]+1
                except:
                    count[k]=1
    
    for i in step_question:
        print(i[0])
    
        if i[0] in next_steps:
            n=next_step(count,directions_dict)
            print("n",n)
            step_ans.delete(1.0,tk.END)
            step_ans.insert(tk.END,n)
            # print(next_step(count))
            break
        elif i[0] in prev_steps:
            p=prev_step(count,directions_dict)

            step_ans.delete(1.0,tk.END)
            step_ans.insert(tk.END,p)
            # print(prev_step(count))
            break
        else:
            
            step_ans.delete(1.0,tk.END)
            step_ans.insert(tk.END,"please again ask question.")
    

def max_match(count):
    # print(count)
    
    return max(count , key = count.get)

def next_step(count,directions_dict):
    k=max_match(count)
    # print(k)
    # print(directions_dict[k+1])
    try:
        return directions_dict[k+1]
    except:
        return "This seems to be last step"

def prev_step(count,directions_dict):
    k=max_match(count)
    try:
        return directions_dict[k-1]
    except:
        return "this seems to be first step"











root=Tk()
root.geometry("1500x720")
bg=tk.PhotoImage(file = "bg.png")
label1 = tk.Label( root, image = bg)
label1.place(x=0,y=0)

Enter_Recipe =Label(root,text="Enter Receipe : ")
Enter_Recipe.grid(row=1,column=2,pady=10)

Entry_recipe=Entry(root,width=75)
Entry_recipe.grid(row=2,column=2,pady=10)

btn_search_list = Button(root,text="Search",command=show_link,width=75)
btn_search_list.grid(column=20,row=2,pady=10)



# dropdown_searched_list.grid(row=2,column=5,pady=10)
clicked=StringVar()

find_btn=Button(root,text="find_Ingredient",command=get_recipe,width=75)
find_btn.grid(row=3,column=20,pady=10)

find_btn_steps=Button(root,text="find_Ingredient",command=get_steps,width=75)
find_btn_steps.grid(row=4,column=20,pady=10)

ingredient_ans=Text(root,width=75)
ingredient_ans.grid(row=5,column=2,pady=10)

step_ans=Text(root,width=75)
step_ans.grid(row=5,column=20,pady=10)

Ask_ingredient=Label(root,text="Ask Ingredient Question: ")
Ask_ingredient.grid(row=13,column=1,pady=10)

Ask_steps=Label(root,text="Ask Steps Question: ")
Ask_steps.grid(row=13,column=19,pady=10)


ingredient_q=Entry(root,width=75)
ingredient_q.grid(row=13,column=2,pady=10)

step_q=Entry(root,width=75)
step_q.grid(row=13,column=20,pady=10)

ingredient_btn=Button(root,text="Ask Ingredient",command=get_recipe)
ingredient_btn.grid(row=16,column=2,pady=10)

step_btn=Button(root,text="Ask Steps",command=get_steps)
step_btn.grid(row=16,column=20,pady=10)









# links_from_web=query_fun(Entry_recipe.get().lower())





root.mainloop()

C:\Users\hp\AppData\Local\Temp\ipykernel_23980\3012056932.py:275: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


https://www.food.com/recipe/greek-style-turkey-burgers-13285
https://www.food.com/recipe/hamburgers-10428
https://www.food.com/recipe/the-perfect-burger-92021
https://www.food.com/recipe/kittencals-juicy-hamburger-burger-208583
https://www.food.com/recipe/almost-white-castle-hamburgers-109264
https://www.food.com/recipe/hamburgers-with-brown-gravy-total-comfort-food-aka-meat-cakes-32973
https://www.food.com/recipe/tasty-turkey-burgers-20731
https://www.food.com/recipe/jalapeno-popper-burgers-29251
https://www.food.com/recipe/tuna-patty-burgers-28025
https://www.food.com/recipe/whole-wheat-hamburger-and-hot-dog-buns-bread-machine-169916
quantity of salt is 1⁄4teaspoon
what
be
next
n when you make the patties place an indention in the middle of each one with your thumb (almost all the way through).  this keeps the burgers from shrinking.  we like ours thick and i like them to stay that way!
what
be
previous
https://www.food.com/recipe/my-familys-favorite-sloppy-joes-pizza-joes-63689
http

In [ ]:
# nltk.download()

In [ ]:
pd.set_option('display.max_colwidth', None)

In [6]:
query ="https://www.food.com/search/"

dish = input("Enter the dish: ")
query = query + dish
query

driver = webdriver.Chrome('./chromedriver') 
driver.get(query)

time.sleep(5) 
html = driver.page_source
links=[]
soup=BeautifulSoup(html,'html.parser')
for a in soup.find_all('h2',class_='title'):
    
    links.append(a.find('a').get('href'))
links

C:\Users\hp\AppData\Local\Temp\ipykernel_22376\3058887810.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


['https://www.food.com/recipe/my-familys-favorite-sloppy-joes-pizza-joes-63689',
 'https://www.food.com/recipe/pizza-dough-for-thin-crust-pizza-70165',
 'https://www.food.com/recipe/ultimate-pizza-sauce-114392',
 'https://www.food.com/recipe/easy-peezy-pizza-dough-bread-machine-pizza-dough-35805',
 'https://www.food.com/recipe/low-carb-pizza-84516',
 'https://www.food.com/recipe/pizza-hut-style-pizza-dough-bread-machine-51209',
 'https://www.food.com/recipe/crazy-crust-pizza-29002',
 'https://www.food.com/recipe/pizza-dough-43740',
 'https://www.food.com/recipe/easy-pizza-pasta-casserole-oamc-111734',
 'https://www.food.com/recipe/quick-and-easy-pizza-dough-117532']